In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape,add
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout,GRU,GaussianNoise, Bidirectional
from autopool import AutoPool1D
from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM


In [ ]:
DATASET_INDEX = 11

MAX_TIMESTEPS = MAX_TIMESTEPS_LIST[DATASET_INDEX]
MAX_NB_VARIABLES = MAX_NB_VARIABLES[DATASET_INDEX]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX]

TRAINABLE = True


In [ ]:
# ''' The adapted SE Layer for 1D sequential data'''
def squeeze_excite_block(input):
    ''' Create a squeeze-excite module
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] 

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    #dimensionality reduction
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    #dimensionality expansion
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    # generating channel statistics
    se = multiply([input, se])
    return se

# ST-DeepHAR

In [ ]:
def create_ST_DeepHAR():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    stride = 3
    
   # ''' The First Path: An LSTM layer augmented with Attention meachanism to learn temporal data representation'''
    x = Masking()(ip)
    x = AttentionLSTM(16)(x)
    x = Dropout(0.8)(x)

    n_feature_maps = 64

    # ''' The Second Path: A modified residual block is proposed to learn spatial data representation'''

    pre = Permute((2, 1))(ip)
    pre = Conv1D(filters=n_feature_maps,kernel_size=8, padding='same', kernel_initializer='he_uniform')(pre)
    pre = BatchNormalization()(pre)
    pre = Activation('relu')(pre)
    
    # '''The RESIDUAL BLOCK 1'''
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=8,padding='same', kernel_initializer='he_uniform')(pre)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.3)(conv_x)
    conv_x = Conv1D(filters=n_feature_maps*2, kernel_size=5,padding='same', kernel_initializer='he_uniform')(conv_x)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.3)(conv_x)
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=3,padding='same', kernel_initializer='he_uniform')(conv_x)
    pre= AutoPool1D()(pre)
    output_block_1 = add([conv_x, pre])
    # '''The Adapted SE BLOCK 1'''
    output_block_1 = squeeze_excite_block(output_block_1)
    
    # '''The RESIDUAL BLOCK 2'''
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=8,padding='same', kernel_initializer='he_uniform')(output_block_1)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.3)(conv_x)
    conv_x = Conv1D(filters=n_feature_maps*2, kernel_size=5,padding='same', kernel_initializer='he_uniform')(conv_x)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.3)(conv_x)
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=3, padding='same', kernel_initializer='he_uniform')(conv_x)
    pre= AutoPool1D()(output_block_1)
    output_block_2 = add([conv_x, pre])
    
    # '''The Adapted SE BLOCK 2'''
    output_block_2 = squeeze_excite_block(output_block_2)
   
    # '''The RESIDUAL BLOCK 3'''
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=8, padding='same', kernel_initializer='he_uniform')(output_block_2)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.3)(conv_x)
    conv_x = Conv1D(filters=n_feature_maps*2, kernel_size=5, padding='same', kernel_initializer='he_uniform')(conv_x)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.3)(conv_x)
    conv_x = Conv1D(filters=n_feature_maps, kernel_size=3, padding='same', kernel_initializer='he_uniform')(conv_x)
    pre= AutoPool1D()(output_block_2)
    output_block_3 = add([conv_x, pre])
    
    # '''The Adapted SE BLOCK 3'''
    output_block_3 = squeeze_excite_block(output_block_3)

    y = BatchNormalization()(output_block_3)
    y = Activation('relu')(y)
    y = GlobalAveragePooling1D()(y)
    
    # '''Concatenation of the Temporal representation(i.e. generated from first path) and Spatial representation (i.e. generated from first path)'''
    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [ ]:
if __name__ == "__main__":
    model = create_ST_DeepHAR()

    train_model(model, DATASET_INDEX, dataset_prefix='har', epochs=200, batch_size=200)

    evaluate_model(model, DATASET_INDEX, dataset_prefix='har', batch_size=200)